In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import lightgbm as lgbm

from scipy.optimize import minimize

In [3]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [4]:
# 로컬 버전

# data_dir = Path('../input/dankook')
# feature_dir = Path('../output/feature')
# val_dir = Path('../output/oof_pred')
# test_dir = Path('../output/test_pred')
# sub_dir = Path('../output/sub')


# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [5]:
# 코렙 

from google.colab import drive
drive.mount('/content/drive')

data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

Mounted at /content/drive


In [6]:
algorithm_name = 'lgbmcv'
feature_name = 'stacking1'
model_name = f'{algorithm_name}_{feature_name}_5'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_target_file = feature_dir / f'feature_target.csv'

stacking1_oof_pred_file = val_dir / f'{model_name}_oof_pred.csv'
stacking1_test_pred_file = test_dir / f'{model_name}_test_pred.csv'
stacking1_submission_file = sub_dir / f'{model_name}_submission.csv'

In [7]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

# Stacking Feature 생성

In [8]:
def load_data(model_names, oof_list, test_list, feature_names=None,number_of_versions=None):
    if number_of_versions == None or number_of_versions == 1:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2']
    elif number_of_versions == 2:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver2.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver2.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2',
                                  f'{model}_ver2_class0',f'{model}_ver2_class1',f'{model}_ver2_class2']
    elif number_of_versions == 2.1:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver3.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver3.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2',
                                  f'{model}_ver3_class0',f'{model}_ver3_class1',f'{model}_ver3_class2']
    elif number_of_versions == 3:
        for model in model_names:
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver1.csv',delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver2.csv', delimiter=','))
            oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred_ver3.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver1.csv',delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver2.csv', delimiter=','))
            test_list.append(np.loadtxt(test_dir / f'{model}_test_pred_ver3.csv', delimiter=','))
            if feature_names != None:
                feature_names += [f'{model}_ver1_class0', f'{model}_ver1_class1', f'{model}_ver1_class2',
                                  f'{model}_ver2_class0',f'{model}_ver2_class1', f'{model}_ver2_class2',
                                  f'{model}_ver3_class0', f'{model}_ver3_class1',f'{model}_ver3_class2']

In [9]:
all_oof = []
all_test = []
feature_names = []

model_names = ['lrcv_feature', 'etscv_feature', 'rfcv_feature', 'gbcv_feature','xgbcv_feature','lgbmcv_feature','adacv_feature_2','adacv_feature']
load_data(model_names, all_oof, all_test, feature_names,3)

model_names = ['lrcv_polynomial_feature','rfcv_polynomial_feature','etscv_polynomial_feature','gbcv_polynomial_feature','adacv_polynomial_feature_2','adacv_polynomial_feature']
load_data(model_names,all_oof, all_test,feature_names,3)

model_names = ['xgbcv_polynomial_feature','lgbmcv_polynomial_feature']
load_data(model_names,all_oof, all_test,feature_names,2.1)

all_oof = np.column_stack(all_oof)
all_test = np.column_stack(all_test)

In [10]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['id',target_column]).values.flatten()
y.shape

(319923,)

# Stacking

- 각 oof마다 fold별로 logloos의 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [13]:
lgbm_params ={
    # 'bagging_fraction': 0.9564304609224545,
    'boosting': 'gbdt',
    'class_weight': None,
    'feature_fraction': 0.831793177270788,
    'lambda_l1': 2.336942442746573,
    'lambda_l2': 0.3201894727986991,
    'learning_rate': 0.010261017351007496,
    'max_depth': 21,
    'metric': 'multi_logloss',
    'min_child_weight': 3.3275156381395915e-06,
    'min_data_in_leaf': 14,
    'n_estimators': 1000,
    'n_jobs': -1,
    'num_class': 3,
    'num_leaves': 38,
    'objective': 'multiclass',
    'random_state': 2020,
    'subsample_for_bin': 280000,
    'verbosity': -1,
 }

 # bagging_fraction 파라미터가 설정되어 있지만, 실제로는 사용하지 않았음...
 # bagging_freq = 1 로 설정해야지 사용이 가능한데...
 # bagging_fraction을 사용하지 않고, 93.835 라는 결과를 낸 것임.
 # hyperopt를 다시 돌려서 튜닝을 해야 할듯...
 # 그리고 그때 나온 결과가 어떻게 되냐에 따라서 어떤 파라미터를 사용할지 결정을 해야 할듯..


 # GPU 버그 있음.. 시발.... GPU 사용 못해 시발
 

In [14]:
mlogloss = []

stk_oof_pred = np.zeros((all_oof.shape[0],num_class))
stk_test_pred = np.zeros((all_test.shape[0],num_class))

kFold = StratifiedKFold(n_splits=n_splits, random_state=2020, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kFold.split(all_oof,y)):
    X_train, X_val = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]

    dtrain = lgbm.Dataset(X_train, label=y_train)
    dval = lgbm.Dataset(X_val, label=y_val)

    lgbm_clf = lgbm.train(params=lgbm_params, train_set=dtrain, num_boost_round=5000,valid_sets=[dtrain,dval], early_stopping_rounds=50, verbose_eval=5000)
    mlogloss.append(lgbm_clf.best_score['valid_1']['multi_logloss'])

    stk_test_pred += lgbm_clf.predict(all_test) / n_splits
    stk_oof_pred[val_idx] = lgbm_clf.predict(X_val)

print('mean logloss= ', np.mean(mlogloss))

Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[1000]	training's multi_logloss: 0.133511	valid_1's multi_logloss: 0.150937
Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[1000]	training's multi_logloss: 0.133823	valid_1's multi_logloss: 0.149652
Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[1000]	training's multi_logloss: 0.133509	valid_1's multi_logloss: 0.15007
Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[1000]	training's multi_logloss: 0.133447	valid_1's multi_logloss: 0.150659
Training until validation scores don't improve for 50 rounds.
Did not meet early stopping. Best iteration is:
[1000]	training's multi_logloss: 0.133509	valid_1's multi_logloss: 0.15039
mean logloss=  0.15034173837685907


# 제출 파일 및 기타 파일 생성

In [15]:
# submission 파일 생성 -> stacking1-lgbm-5---3 과 결과가 동일한 것을 확인함.

sub = pd.read_csv(sample_file)
sub[target_column] = np.argmax(stk_test_pred, axis=1)
sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

In [16]:
# stcking1_oof_pred 파일 생성

np.savetxt(stacking1_oof_pred_file, stk_oof_pred, fmt='%.18f',delimiter=',')

In [17]:
# stacking1_test_pred 파일 생성

np.savetxt(stacking1_test_pred_file, stk_test_pred, fmt='%.18f', delimiter=',')